<a href="https://colab.research.google.com/github/tcbic/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module2-sql-for-analysis/Unit_3_Sprint_2_Module_2_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connecting PostgreSQL from Python**

In [1]:
!pip install psycopg2-binary

     |████████████████████████████████| 2.9MB 4.0MB/s 


In [0]:
import psycopg2

In [0]:
#Input information from ElephantSQL.

dbname = 'ubvnhvje'

user = 'ubvnhvje'

password = 'BLANK'

host = 'raja.db.elephantsql.com'

In [0]:
#Make a connection object (establishing a connection allows us to authenticate and connect to
#the database programmatically).

pg_conn = psycopg2.connect(dbname=dbname , user=user, password=password, host=host)

In [0]:
#Create a cursor (let's us be somewhere in the data to retrieve a row/record).

pg_curs = pg_conn.cursor()

In [6]:
pg_curs.execute('SELECT * FROM test_table;')
pg_curs.fetchall()

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

In [0]:
#So, what we are doing is getting the RPG data out of SQLite and inserting it into PostgreSQL.
#AKA, we are making a data pipeline!
#AKA, an ETL (Extract Transform Load)

In [0]:
#Now, make the character table in PostgreSQL.

In [9]:
#Download the raw sqlite3 database from GitHub.

!wget https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true

--2019-08-14 03:24:17--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-08-14 03:24:17--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-08-14 03:24:17--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module

In [10]:
!ls -alh

total 264K
drwxr-xr-x 1 root root 4.0K Aug 14 03:24  .
drwxr-xr-x 1 root root 4.0K Aug 14 02:51  ..
drwxr-xr-x 1 root root 4.0K Aug  7 16:05  .config
-rw-r--r-- 1 root root 248K Aug 14 03:24 'rpg_db.sqlite3?raw=true'
drwxr-xr-x 1 root root 4.0K Aug  2 16:06  sample_data


In [0]:
#Rename the file.

!mv 'rpg_db.sqlite3?raw=true' rpg_db.sqlite3

In [0]:
#Explore the sqlite3 database we downloaded in python.

In [0]:
#First, import the sqlite3 module.

import sqlite3

In [0]:
#Using sl as abbreviation for sqlite3.

sl_conn = sqlite3.connect('rpg_db.sqlite3')
sl_curs = sl_conn.cursor()

In [15]:
#An example query to check it out.

sl_curs.execute('SELECT COUNT(*) FROM charactercreator_character;').fetchall()

[(302,)]

In [0]:
#Our goal- an ETL/data pipeline 
#Get the character data from SQLite to Python.

#Our target table(for live demo) - characters

#Extract step:

characters = sl_curs.execute('SELECT * FROM charactercreator_character;').fetchall()

In [17]:
#Look at the first character.

characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [18]:
#Look at the last character.

characters[-1]

(302, 'Aliquam n', 0, 0, 10, 1, 1, 1, 1)

In [19]:
#Check number of character entries we should have.

len(characters)

302

In [0]:
#So, our Extract is done! Transform next...
#Now that we have the data, we need some place to put the data. 

In [0]:
#To do this, we need to define the schema. 
#We need the PostgreSQL db to have a table with an appropriate schema.

In [0]:
#Create a table.
#This creates a table within our PostgreSQL database.

create_character_table = """ 
  CREATE TABLE charactercreator_character (
    character_id SERIAL PRIMARY KEY,
    name VARCHAR(30),
    level INT,
    exp INT,
    hp INT,
    strength INT,
    intelligence INT,
    dexterity INT,
    wisdom INT
   );
  """

In [23]:
#Just execute, no fecthall as we are not retrieving anything.

pg_curs.execute(create_character_table)

DuplicateTable: ignored

In [0]:
#The Transform (making the target ready to get data) is done. Now, we need
#to actually load values, insert our characters.

In [0]:
#Reference cells...

print(characters[0])

#Slice to remove the id value.

print(characters[0][1:])

In [0]:
#And instead of being in the form of a tuple, we want it in the form of a string.

str(characters[0][1:])

In [0]:
#Use the cells above as a reference, as those contents look a lot like what
#we want to include in our table.

#This is for one character.

example_insert = """
INSERT INTO charactercreator_charcater
(name, level, exp, hp, strength, intelligence, dexterity, wisdom)
VALUES """ + str(characters[0][1:]) + ';'

In [0]:
print(example_insert)

In [0]:
#That's for one character, but how do we do this for all characters? Loops.

for character in characters:
  insert_character = """
    INSERT INTO charactercreator_character
    (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
    VALUES""" + str(characters[1:]) + ';'

In [0]:
pg_curs.execute(insert_character)

In [0]:
#At this point, we can see it from this cursor- but not from elephantsql.com.

#We have to commit, which saves to the database!

pg_curs.close()
pg_conn.commit()

In [0]:
#Ok, we can see it in elephantsql.com now.
#But, let's also check programmatically.

#Make another cursor as we closed the cursor.
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [0]:
characters[0]

In [0]:
for character, pg_character in zip(characters, pg_characters):
  assert character == pg_character

In [0]:
#If nothing happens, this means they are all the same.
#We have successfully performed and validated an ETL.